In [1]:
#! pip install python-resize-image

In [2]:
#! pip install tensorflow

In [3]:
import pandas as pd
import numpy as np

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, UpSampling2D, Activation, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import image_dataset_from_directory

import tensorflow as tf
from tensorflow.keras.models import model_from_json
import cv2
#from skimage import color
from PIL import Image
#import imageio
#from resizeimage import resizeimage
import os

import pickle

from sklearn.metrics import confusion_matrix

In [4]:
#drive.mount('/content/drive')

In [5]:
#files.upload()

In [6]:
#! mkdir ~/.kaggle
#! cp kaggle.json ~/.kaggle/
#! chmod 600 ~/.kaggle/kaggle.json

In [7]:
#!kaggle datasets download -d validmodel/indo-fashion-dataset

In [8]:
#!unzip -q /content/indo-fashion-dataset.zip


In [9]:
df_train = pd.read_json('/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/data/raw/indian_clothing/train_data.json', lines=True)

In [10]:
df_train.head()

,image_url,image_path,brand,product_title,class_label,color
0,https://m.media-amazon.com/images/I/81XKaSKvly...,images/train/0.jpeg,Womanista,Women's Georgette Saree with Blouse Piece (TKI...,saree,NaN
1,https://m.media-amazon.com/images/I/91dDMDg+QT...,images/train/1.jpeg,Yashika,Women Linen Printed Sarees Free Size (BEE),saree,NaN
2,https://m.media-amazon.com/images/I/71TJ9p46PV...,images/train/2.jpeg,Womanista,Organza Floral Print Saree with Blouse Piece(W...,saree,NaN
3,https://m.media-amazon.com/images/I/81Y+je7CEg...,images/train/3.jpeg,PERFECTBLUE,Women's Digital Cotton Linen Blend Saree with ...,saree,NaN
4,https://m.media-amazon.com/images/I/61B8o9Ulqp...,images/train/4.jpeg,VAIVIDHYAM,Designer Sarees Women's Banarasi Cotton Silk S...,saree,NaN


In [11]:
df_train.drop(['image_url', 'brand', 'product_title', 'color'], axis=1, inplace=True)

In [12]:
df_train.head()

,image_path,class_label
0,images/train/0.jpeg,saree
1,images/train/1.jpeg,saree
2,images/train/2.jpeg,saree
3,images/train/3.jpeg,saree
4,images/train/4.jpeg,saree


In [13]:
df_test = pd.read_json('/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/data/raw/indian_clothing/test_data.json', lines = True)

In [14]:
df_test.head()

,image_url,image_path,brand,product_title,class_label,color
0,https://m.media-amazon.com/images/I/71NZQ-aaAS...,images/test/0.jpeg,TANTUJA BENGAL HANDLOOM,(A Govt. of West Bengal Enterprises) Handloom ...,saree,NaN
1,https://m.media-amazon.com/images/I/81aQmbWsG7...,images/test/1.jpeg,Trilok Fab,Women's Silk Saree With Blouse Piece,saree,NaN
2,https://m.media-amazon.com/images/I/71YjGvzs9y...,images/test/2.jpeg,Nine Sister,Women's Georgette Net Embroidered Saree with B...,saree,NaN
3,https://m.media-amazon.com/images/I/81JYhYV5Xr...,images/test/3.jpeg,Gauri Laxmi Enterprise,Women's Crepe Saree with Blouse (GLE$CHICKOO C...,saree,NaN
4,https://m.media-amazon.com/images/I/91fXc5i5Dy...,images/test/4.jpeg,SOURBH,Women's Cotton Blend Digital Butta Printed Sar...,saree,NaN


In [15]:
df_test.drop(['image_url', 'brand', 'product_title', 'color'], axis=1, inplace=True)

In [16]:
df_test.tail()

,image_path,class_label
7495,images/test/7495.jpeg,women_kurta
7496,images/test/7496.jpeg,women_kurta
7497,images/test/7497.jpeg,women_kurta
7498,images/test/7498.jpeg,women_kurta
7499,images/test/7499.jpeg,women_kurta


In [17]:
#df_test.to_csv('/content/drive/My Drive/capstone_3/df_test.csv')

In [18]:

df_val = pd.read_json('/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/data/raw/indian_clothing/val_data.json', lines=True)

In [19]:
df_val.head()

,image_url,image_path,brand,product_title,class_label,color
0,https://m.media-amazon.com/images/I/81LOPbFPiQ...,images/val/0.jpeg,Generic,Women's Khadi Cotton Saree With Blouse Piece (...,saree,NaN
1,https://m.media-amazon.com/images/I/81Q8Oktw4s...,images/val/1.jpeg,Yashvi Designer,Women's Net Saree With Unstitched Blouse Piece,saree,NaN
2,https://m.media-amazon.com/images/I/613S1YGCmo...,images/val/2.jpeg,Aarrah,Georgette Strip Print Saree[S_SHIKHA30021SR02_...,saree,NaN
3,https://m.media-amazon.com/images/I/819budhQl1...,images/val/3.jpeg,MIMOSA,Women's Patola Style Art Silk Saree (Green),saree,NaN
4,https://m.media-amazon.com/images/I/61cAZ94ZQV...,images/val/4.jpeg,Generic,Women's Pure Hand Block Patola Printed Cotton ...,saree,NaN


In [20]:

df_val.drop(['image_url', 'brand', 'product_title', 'color'], axis=1, inplace=True)

In [21]:
df_val.head()

,image_path,class_label
0,images/val/0.jpeg,saree
1,images/val/1.jpeg,saree
2,images/val/2.jpeg,saree
3,images/val/3.jpeg,saree
4,images/val/4.jpeg,saree


In [22]:
df_train = pd.get_dummies(data=df_train, columns=['class_label'])

In [23]:
df_train.head()

,image_path,class_label_blouse,class_label_dhoti_pants,class_label_dupattas,class_label_gowns,class_label_kurta_men,class_label_leggings_and_salwars,class_label_lehenga,class_label_mojaris_men,class_label_mojaris_women,class_label_nehru_jackets,class_label_palazzos,class_label_petticoats,class_label_saree,class_label_sherwanis,class_label_women_kurta
0,images/train/0.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,images/train/1.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,images/train/2.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,images/train/3.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,images/train/4.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [24]:
df_val = pd.get_dummies(data=df_val, columns=['class_label'])

In [25]:
df_val.head()

,image_path,class_label_blouse,class_label_dhoti_pants,class_label_dupattas,class_label_gowns,class_label_kurta_men,class_label_leggings_and_salwars,class_label_lehenga,class_label_mojaris_men,class_label_mojaris_women,class_label_nehru_jackets,class_label_palazzos,class_label_petticoats,class_label_saree,class_label_sherwanis,class_label_women_kurta
0,images/val/0.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
1,images/val/1.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,images/val/2.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,images/val/3.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
4,images/val/4.jpeg,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0


In [26]:
df_test = pd.get_dummies(data=df_test, columns=['class_label'])

In [27]:
df_train.columns

Index(['image_path', 'class_label_blouse', 'class_label_dhoti_pants',
       'class_label_dupattas', 'class_label_gowns', 'class_label_kurta_men',
       'class_label_leggings_and_salwars', 'class_label_lehenga',
       'class_label_mojaris_men', 'class_label_mojaris_women',
       'class_label_nehru_jackets', 'class_label_palazzos',
       'class_label_petticoats', 'class_label_saree', 'class_label_sherwanis',
       'class_label_women_kurta'],
      dtype='object')

In [40]:
# Creating generator images sizes 64 x 64
columns = ['class_label_blouse', 'class_label_dhoti_pants',
       'class_label_dupattas', 'class_label_gowns', 'class_label_kurta_men',
       'class_label_leggings_and_salwars', 'class_label_lehenga',
       'class_label_mojaris_men', 'class_label_mojaris_women',
       'class_label_nehru_jackets', 'class_label_palazzos',
       'class_label_petticoats', 'class_label_saree', 'class_label_sherwanis',
       'class_label_women_kurta']
train_datagen=ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_dataframe(dataframe=df_train, directory='/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/data/raw/indian_clothing/', x_col='image_path', y_col=columns, target_size=(64,64), batch_size=128, seed=26, save_to_dir='/Users/butler/Documents/untitled/', class_mode='raw')

Found 91166 validated image filenames.


In [41]:
columns = ['class_label_blouse', 'class_label_dhoti_pants',
       'class_label_dupattas', 'class_label_gowns', 'class_label_kurta_men',
       'class_label_leggings_and_salwars', 'class_label_lehenga',
       'class_label_mojaris_men', 'class_label_mojaris_women',
       'class_label_nehru_jackets', 'class_label_palazzos',
       'class_label_petticoats', 'class_label_saree', 'class_label_sherwanis',
       'class_label_women_kurta']
val_datagen=ImageDataGenerator(rescale=1./255)
val_generator = val_datagen.flow_from_dataframe(dataframe=df_val, directory='/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/data/raw/indian_clothing/', x_col='image_path', y_col=columns, target_size=(64,64), batch_size=128, seed=26, save_to_dir='/Users/butler/Documents/untitled/', class_mode='raw')

Found 7500 validated image filenames.


In [28]:
columns = ['class_label_blouse', 'class_label_dhoti_pants',
       'class_label_dupattas', 'class_label_gowns', 'class_label_kurta_men',
       'class_label_leggings_and_salwars', 'class_label_lehenga',
       'class_label_mojaris_men', 'class_label_mojaris_women',
       'class_label_nehru_jackets', 'class_label_palazzos',
       'class_label_petticoats', 'class_label_saree', 'class_label_sherwanis',
       'class_label_women_kurta']
test_datagen=ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_dataframe(dataframe=df_test, directory='/content/', x_col='image_path', y_col=columns, target_size=(64,64), batch_size=128, seed=26, save_to_dir='/Users/butler/Documents/untitled2/', class_mode='raw')

Found 0 validated image filenames.


/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/keras/preprocessing/image.py:1137: UserWarning: Found 7500 invalid image filename(s) in x_col="image_path". These filename(s) will be ignored.
  warnings.warn(


In [42]:
# Setting up model
img_width, img_height = 64, 64

train_data = train_generator
val_data = val_generator
nb_train_samples = 91166
nb_validation_samples = 7500
epochs = 500
batch_size = 64
n_classes = 15

input_shape = (img_width, img_height, 3)

In [43]:

model = Sequential()

model.add(Conv2D(32, (3,3), activation='relu', input_shape=input_shape))
model.add(Conv2D(32, (3,3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))
model.add(Conv2D(64, (3,3), activation='relu'))

model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(32, activation='relu'))
model.add(Dropout(0.35))
model.add(Dense(n_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [44]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath='/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models',
        save_best_only=True,
        monitor='val_accuracy'
    )
]

In [45]:
history = model.fit(
    x=train_data,
    y=None,
    validation_data= val_data,
    epochs=epochs,
    verbose="auto",
    callbacks=callbacks,
    validation_split=0.0,
    shuffle=True,
    validation_freq=10,
    )

Epoch 1/500
163/713 [=====>........................] - ETA: 2:23:31 - loss: 2.0737 - accuracy: 0.3408

2023-09-01 18:25:28.560121: W tensorflow/core/framework/op_kernel.cc:1816] UNKNOWN: OSError: [Errno 28] No space left on device
Traceback (most recent call last):

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/PIL/ImageFile.py", line 518, in _save
    fh = fp.fileno()

AttributeError: '_idat' object has no attribute 'fileno'


During handling of the above exception, another exception occurred:


Traceback (most recent call last):

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/PIL/Image.py", line 2431, in save
    save_handler(self, fp, filename)

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/PIL/PngImagePlugin.py", line 1420, in _save
    ImageFile._save(im, _idat(fp, chunk), [("zip", (0, 0) + im.size, 0, rawmode)])

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/PIL/ImageFile.py", line 522, in _save
    _encode_tile(im, fp, tile, bufsize, None, exc)

  File "/Users/butler/anaconda3/envs/CNN/lib/

UnknownError: Graph execution error:

OSError: [Errno 28] No space left on device
Traceback (most recent call last):

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/PIL/ImageFile.py", line 518, in _save
    fh = fp.fileno()

AttributeError: '_idat' object has no attribute 'fileno'


During handling of the above exception, another exception occurred:


Traceback (most recent call last):

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/PIL/Image.py", line 2431, in save
    save_handler(self, fp, filename)

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/PIL/PngImagePlugin.py", line 1420, in _save
    ImageFile._save(im, _idat(fp, chunk), [("zip", (0, 0) + im.size, 0, rawmode)])

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/PIL/ImageFile.py", line 522, in _save
    _encode_tile(im, fp, tile, bufsize, None, exc)

  File "/Users/butler/anaconda3/envs/CNN/lib/python3.9/site-packages/PIL/ImageFile.py", lin

In [46]:
scores = model.evaluate(x=val_generator, y=None, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

2023-08-31 18:31:25.501599: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


KeyboardInterrupt: 

In [47]:
model.save('/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models/model_12.keras')

In [45]:
tf.keras.saving.save_model(
    model, '/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models/model_12a', overwrite=True, save_format=None)

2023-08-31 18:31:00.579246: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,32]
	 [[{{node inputs}}]]
2023-08-31 18:31:00.675646: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,32]
	 [[{{node inputs}}]]


INFO:tensorflow:Assets written to: /Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models/model_11a/assets


INFO:tensorflow:Assets written to: /Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models/model_11a/assets


In [41]:
#!pip install h5py
# serialize model to JSON
loaded_model_json = loaded_model.to_json()
with open('/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models/model20230831.json', 'w') as json_file:
    json_file.write(loaded_model_json)
# serialize weights to HDF5
loaded_model.save_weights('/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models/model20230831.h5')

In [30]:
json_file = open('/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models/model20230831.json', 'r')
loaded_model2_json = json_file.read()
json_file.close()
loaded_model2 = model_from_json(loaded_model2_json)

In [31]:
loaded_model2.load_weights('/Users/butler/Documents/hp/Retrieval_of_Indian_clothing_images/models/model20230831.h5')
print("Loaded model from disk")

Loaded model from disk
